In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 17:34:18 2020

@author: Guillermo Sánchez Gutiérrez-Cabello
"""
#%%
import numpy as np
import pandas as pd
import os
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import nltk
nltk.download("popular") # required to download the stopwords lists
from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')
#%%
mainpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetOriginales"

def dfFix(df,col1 = False,col2 = False):
    result = df.copy()
    if(col1):
        x = result.columns.get_loc(col1)
        result.drop(result.columns[0:x],axis = 1, inplace = True)
    if(col2):
        y = result.columns.get_loc(col2)
        result.drop(result.columns[y:],axis = 1, inplace = True)
    return result

def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def dropRow(df,i):
    return df.drop(index = i)

def mkCSV(df,fileName):
    df = df.dropna(how = 'all')
    df *= 1  #Cambia columnas Booleanas por [0,1] y el resto de datos los mantiene igual.
    #df = df.fillna(-1)   
    fileName = fileName.lower()
    df.to_csv('DataSetFinales/'+fileName,sep=',',header = False, index=False, encoding='utf-8') #Header e index a false para no mostrarlo en el csv
    
def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

def fixBibliography(df):
    df = dfFix(df,"GENERAL INFORMATION - COUNTRY LEVEL")
    df.columns = ['GeneralInfo', 'CommunityCountry', 'RefugeeCountry']
    df.set_index('GeneralInfo', inplace = True)
    df = df.transpose()
    df.reset_index(inplace = True)
    return df

def getSubColumnNames(df,x):
    columns = df.columns
    array = []
    for column in columns:
        column = column[x:]
        array.append(column)
    return pd.DataFrame(array) 

def addInstitutionAndType(df,array1,array2,instType,index):
    refugees = dropRow(df,index)
    refugees = refugees.dropna(axis = 1)
    refugees = np.array(refugees)
    for row in refugees:
        for elem in row:
            array1 = np.append(array1,elem)
            array2 = np.append(array2,instType)
    return array1,array2

def politicalActor(df1,df2,df3,df4,df5,index):
    institution = []
    instType = []

    institution, instType  = addInstitutionAndType(df1,institution,instType,'Public Institution',index)
    institution, instType  = addInstitutionAndType(df2,institution,instType,'Private Institution',index)
    institution, instType  = addInstitutionAndType(df3,institution,instType,'NGO',index)
    institution, instType  = addInstitutionAndType(df4,institution,instType,'International Agency',index)
    institution, instType  = addInstitutionAndType(df5,institution,instType,'Local',index)

    institution = pd.DataFrame(institution)
    institution = institution.reset_index(drop = True)
    instType = pd.DataFrame(instType)
    instType = instType.reset_index(drop = True)
    
    return concatDF(institution,instType)

def get_claveValor(df1,df2):
    array1 = np.array(df2)
    array2 = np.array(df1)
    result1 = []
    result2 = []
    i = 0
    for row in array1:
        var = array2[i]
        for elem in row:
            result1 = np.append(result1,elem)
            result2 = np.append(result2,var)
        i+=1
    result2 = pd.DataFrame(result2)
    result2 = result2.reset_index(drop = True)
    result1 = pd.DataFrame(result1)
    result1 = result1.reset_index(drop = True)
    return concatDF(result2,result1)

def get_FSClaveValor(df1,df2):
    df2 = df2.transpose()
    array = np.array(df2)
    array2 =[]
    i = 0
    for row in array:
        for elem in row:
            array2 = np.append(array2,elem)
        i+=1  
    df2 = pd.DataFrame(array2)
    return concatDF(df1,df2)

def get_valueBySector(df1,df2):
    df2 = df2.reset_index()
    array1 = np.array(df1)
    i = 0
    for row in array1:
        for elem in row:
            if(elem == False):
                df2 = dropRow(df2,i)
        i += 1
    df2 = df2.set_index('index')
    return df2

def separateValues(df):
    array = np.array(df)
    corpus = []
    for row in array:
        for elem in row:
            corpus = np.append(corpus,[elem])
    X = count_vectorizer.fit_transform(corpus)
    array = count_vectorizer.get_feature_names()
    return pd.DataFrame(array)  

def vectorizeValue(df):
    df = separateValues(df)
    year = np.array(['january','february','march','april','may','june','july','august','september','october','november','december'])
    result = np.array([],dtype = bool)
    df = np.array(df)
    for elem in year:
        flag = False
        for column in df:
            for month in column:
                if(column == elem):
                    flag = True
        if(flag):
            result = np.append(result,True)
        else:
            result = np.append(result,False)
    return pd.DataFrame(result)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
#%% CSV to DataFrame
Bibliography = pd.read_excel(getPath(mainpath,"Bibliography_120220.xlsx"))
Bibliography = fixBibliography(Bibliography)
Entities = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Entities_Interview_results.csv"))
LocalLeaders = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Local_leaders_v3_results.csv"))
HouseHold = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Survey_household_v6_results.csv"))
WomenGroup = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Women_Focus_Group2_results.csv"))
SanitationInfra = pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv"))
Priorities = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Priorities_v3_results.csv"))
GeneralForm = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_form_v3_results.csv"))
PublicSpace = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Public_Space_results.csv"))
WaterInf = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Water_Infrastructure_results.csv"))
SanitationInf = pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv"))
WasteManagementInf = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Waste_Management_Infrastructure_results.csv"))
EnergyINF = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Energy_Infrastructure_results.csv"))
Business = pd.read_csv(getPath(mainpath,"NAUTIA1_0_Business_surveys_v3_results.csv"))
MobilityINF = pd.read_csv(getPath(mainpath,"NAUTIA_1_0__Transport_servicesaccess_points_results.csv")) 
ComunalServices = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Communal_Services_results.csv")) 
GeneralCitizen = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_Citizen_Focus_Group_results.csv"))
Shelter = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Shelter_results.csv"))
FarmyardCrop = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Farmyard_and_Crops_results.csv"))

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [6]:
Bibliography.shape

(2, 192)

In [7]:
Entities.shape

(5, 72)

In [8]:
LocalLeaders.shape

(8, 22)

In [48]:
HouseHold.shape

(29, 42)

In [10]:
WomenGroup.shape

(4, 27)

In [11]:
SanitationInfra.shape

(43, 14)

In [12]:
Priorities.shape

(5, 88)

In [113]:
GeneralForm.shape

(1, 19)

In [14]:
PublicSpace.shape

(9, 14)

In [15]:
WaterInf.shape

(38, 13)

In [16]:
SanitationInf.shape

(43, 14)

In [17]:
WasteManagementInf.shape

(3, 10)

In [18]:
EnergyINF.shape

(12, 38)

In [38]:
Business.shape

(21, 25)

In [20]:
MobilityINF.shape

(5, 10)

In [114]:
ComunalServices.head(5)

,start,end,General_Information:Type_of_setlement,General_Information:Type_of_Host_Community,General_Information:Record_your_current_location:Latitude,General_Information:Record_your_current_location:Longitude,General_Information:Record_your_current_location:Altitude,General_Information:Record_your_current_location:Accuracy,General_Information:Type_of_service,General_Information:Other_service,General_Information:Name,General_Information:Sharing_Services,Grain_Mill_Details:Available,Grain_Mill_Details:Type,Grain_Mill_Details:Engine,Grain_Mill_Details:Adecuate_Construction,Cementary_Details:Drainage,Cementary_Details:Height,Education_level,education_details:Student,education_details:Workers,education_details:Teachers,education_details:Materials,education_details:Subjects,education_details:Subjects_001,education_details:Subject_needed,education_details:Start_001,education_details:Finish,Health_Center,Health_Center_details:Capacity,Associate_infrastructure:Sanitation,Associate_infrastructure:Important_Note,Associate_infrastructure:Water,Associate_infrastructure:Important_Note_001,Associate_infrastructure:Energy,Energy_Details:Energy_Source,Energy_Details:Type_of_water_supply,Energy_Details:Generator_Type,Energy_Details:Electrical_Details,Energy_Details:Electrical_Appliances:Devices,Energy_Details:Electrical_Appliances:Lantern_details:Amount,Energy_Details:Electrical_Appliances:Lantern_details:Power,Energy_Details:Electrical_Appliances:Lamp:Amount_001,Energy_Details:Electrical_Appliances:Lamp:Power_001,Energy_Details:Electrical_Appliances:Radio:Amount_002,Energy_Details:Electrical_Appliances:Radio:Power_002,Energy_Details:Electrical_Appliances:TV_or_DVD:Amount_003,Energy_Details:Electrical_Appliances:TV_or_DVD:Power_003,Energy_Details:Electrical_Appliances:PC_or_Tablet:Amount_004,Energy_Details:Electrical_Appliances:PC_or_Tablet:Power_004,Energy_Details:Electrical_Appliances:Mobile_Details:Amount_005,Energy_Details:Electrical_Appliances:Mobile_Details:Power_005,Energy_Details:Electrical_Appliances:Electronic_Cooker:Amount_006,Energy_Details:Electrical_Appliances:Electronic_Cooker:Power_006,Energy_Details:Electrical_Appliances:Fridge_Details:Amount_007,Energy_Details:Electrical_Appliances:Fridge_Details:Power_007,Energy_Details:Electrical_Appliances:Other_Details:Amount_008,Energy_Details:Electrical_Appliances:Other_Details:Power_008,Construction_Details:Appropiate_Roof,Construction_Details:Structural_Security,Construction_Details:Resilient,Construction_Details:Thermal_Confort,meta:instanceID
0,2019-11-18 10:45:24.364,2019-11-18 18:46:50.261,refugee_camp,NaN,NaN,NaN,NaN,NaN,educational_center,NaN,Anda Mohamed,Refugees,NaN,NaN,NaN,NaN,NaN,NaN,secundary,985.0,16.0,33.0,books__noteboo_2,mathematic natural_scienc social_science geogr...,NaN,Música,08:00:00,13:45:00,NaN,NaN,yes,NaN,yes,NaN,yes,electrical_gri,no,NaN,yes,lamp pc_or_tablet other,NaN,NaN,64.0,50.0,NaN,NaN,NaN,NaN,20.0,350.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2000.0,yes,yes,no,yes,uuid:63fdce5a-8df6-4980-8c42-f02a00350a87
1,2019-11-17 21:48:18.876,2019-11-18 18:52:07.268,refugee_camp,NaN,NaN,NaN,NaN,NaN,educational_center,NaN,Ali Mabrik Buzia,Refugees,NaN,NaN,NaN,NaN,NaN,NaN,nursery,338.0,4.0,15.0,notebook_and_p,NaN,NaN,NaN,08:00:00,12:00:00,NaN,NaN,yes,NaN,yes,NaN,yes,electrical_gri,no,NaN,yes,lamp,NaN,NaN,16.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,yes,uuid:dd6626d6-4f00-4971-b8d1-060cd8bf78e2
2,2019-11-18 09:47:21.552,2019-11-18 23:35:32.996,refugee_camp,NaN,NaN,NaN,NaN,NaN,educational_center,NaN,Bala Ahmed zein,Refugees,NaN,NaN,NaN,NaN,NaN,NaN,primary,715.0,15.0,22.0,books__noteboo,mathematic language natural_scienc social_scie...,NaN,Música,07:00:00,12:15:00,NaN,NaN,yes,NaN,no,NaN,yes,electrical_gri,no,NaN,yes,lamp tv_or_dvd fridge,NaN,NaN,40.0,50.0,NaN,NaN,1.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,885.0,NaN,NaN,yes,yes,no,yes,uuid:ae16ddfc-fef4-4677-8a6e-555a17e4c50c
3,2019-11-20 09:57:31.904,2019-11-20 14:32:12.877,refugee_camp,NaN,27.496950,-7.8231

In [22]:
GeneralCitizen.shape

(4, 43)

In [23]:
Shelter.shape

(28, 27)

In [24]:
FarmyardCrop.shape

(22, 18)

In [28]:
PublicSpace.shape

(9, 14)

In [85]:
def set_sector(df,sect,concat = True):
    sector = np.array([])
    df = df.dropna(how = 'all')
    df = np.array(df)
    for column in df:
        sector = np.append(sector,sect)
    sector = pd.DataFrame(sector)
    df = pd.DataFrame(df)
    if(concat):
        result = concatDF(sector,df)
    else:
        result = sector
    return result      

In [148]:
df1 = dfFix(ComunalServices,"Health_Center","Health_Center_details:Capacity")
df1 = df1.isin(["hospital"])                                                  #probar con datos
df2 = dfFix(ComunalServices,"General_Information:Record_your_current_location:Latitude","General_Information:Record_your_current_location:Accuracy")
df3 = dfFix(ComunalServices,"Health_Center_details:Capacity","Associate_infrastructure:Sanitation")
S_Hospital = concatDF(df2,df3)
S_Hospital = get_valueBySector(df1,S_Hospital)
mkCSV(S_Hospital,"S_Hospital.csv")


In [149]:
S_Hospital

,0,1,2,3
index,,,,
28,NaN,NaN,NaN,30.0
29,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,20.0
